In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('amazon_baby.csv')
df['sentiments'] = df.rating.apply(lambda x: 0 if x in [1, 2] else 1)

In [3]:
df.head(5)

,name,review,rating,sentiments
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,1
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1


In [11]:
import numpy as np
import tensorflow
from tensorflow import keras
from keras import preprocessing
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
tokenizer = Tokenizer(oov_token='<OOV>')

In [10]:
split = round(len(df)*0.8)
train_reviews = df['review'][:split]
train_label = df['sentiments'][:split]
test_reviews = df['review'][split:]
test_label = df['sentiments'][split:]

In [12]:
training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []
for row in train_reviews:
    training_sentences.append(str(row))
for row in train_label:
    training_labels.append(row)
for row in test_reviews:
    testing_sentences.append(str(row))
for row in test_label:
    testing_labels.append(row)

In [13]:
vocab_size = 40000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'
padding_type = 'post'

In [14]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

In [15]:
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
testing_sentences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sentences, maxlen=max_length)

In [17]:

model = tensorflow.keras.Sequential([
    tensorflow.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tensorflow.keras.layers.GlobalAveragePooling1D(),
    tensorflow.keras.layers.Dense(6, activation='relu'),
    tensorflow.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 16)           640000    
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 6)                 102       
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 640,109
Trainable params: 640,109
Non-trainable params: 0
_________________________________________________________________


In [19]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)
history = model.fit(padded, training_labels_final, epochs=20, validation_data=(testing_padded, testing_labels_final))

Epoch 1/20
4589/4589 [==============================] - 18s 4ms/step - loss: 0.2920 - accuracy: 0.8832 - val_loss: 0.2144 - val_accuracy: 0.9160
Epoch 2/20
4589/4589 [==============================] - 17s 4ms/step - loss: 0.2088 - accuracy: 0.9143 - val_loss: 0.2010 - val_accuracy: 0.9164
Epoch 3/20
4589/4589 [==============================] - 18s 4ms/step - loss: 0.1922 - accuracy: 0.9212 - val_loss: 0.1943 - val_accuracy: 0.9199
Epoch 4/20
4589/4589 [==============================] - 18s 4ms/step - loss: 0.1822 - accuracy: 0.9255 - val_loss: 0.1955 - val_accuracy: 0.9200
Epoch 5/20
4589/4589 [==============================] - 18s 4ms/step - loss: 0.1742 - accuracy: 0.9289 - val_loss: 0.2001 - val_accuracy: 0.9181
Epoch 6/20
4589/4589 [==============================] - 18s 4ms/step - loss: 0.1676 - accuracy: 0.9314 - val_loss: 0.2014 - val_accuracy: 0.9172
Epoch 7/20
4589/4589 [==============================] - 18s 4ms/step - loss: 0.1604 - accuracy: 0.9340 - val_loss: 0.2050 - val_ac